## Finding the Center of Mass of the Police Districts

* In this notebook we find the centers of mass for each of the police districts, represented by polygons.

In [1]:
import os
import time
from tqdm import tqdm

import gmaps
import shapely
import pandas as pd
import geopandas as gpd

In [2]:
# Set up API Authentication.
key_path = "/Users/administrator/Documents/Projects/abq_crime/api_key.txt"
with open(key_path) as f:
    api_key = f.readline()
    f.close()

# Configure Google Maps API.
gmaps.configure(api_key=api_key)

In [3]:
shapefile_path = "/Users/administrator/Documents/Projects/sf-crime-exploration/data/PdDistricts"
districts_dataframe = gpd.read_file(shapefile_path)
districts_dataframe = districts_dataframe[["district", "geometry"]]

### Splitting MultiPolygons

* In this section we split up multiploygons that represent a single police district.
* We add the new polygons as entries in the dataframe and delete the original one.

In [4]:
# Find the indices of entries that contain a multipolygon in the geometry column.
mpolygon_indices = [type(item) == shapely.geometry.multipolygon.MultiPolygon for item in districts_dataframe["geometry"]]
mpolygon_indices = [idx for idx, item in enumerate(mpolygon_indices) if item == True]

In [5]:
# For the entires that contain a multipolygon, split up the polygons.
new_entries = list()
for index in mpolygon_indices:
    district = districts_dataframe.iloc[index]["district"]
    for poly in districts_dataframe.iloc[index]["geometry"]:
        new_entries.append((district, poly))

In [6]:
# Add these new entries into the dataframe, and delete the original ones.
for entry in new_entries:
    districts_dataframe.loc[len(districts_dataframe)] = entry

# Delete the original entries.
districts_dataframe.drop(mpolygon_indices, axis=0, inplace=True)
districts_dataframe.reset_index(drop=True, inplace=True)

### Converting from (Longitude, Latitude) to (Latitude, Longitude)

* In this section we convert the vertices of the polygons from the (longitude, latitude) convention to the (latitude, longitude) convention.
* This conversion is done so that the rest of the notebook is consistent, as most functions assume the (latitude, longitude) convention especially the Google Maps API.

In [7]:
def convert_coordinates(polygon):
    """Converts coordinates from (longitude, latitude) to (latitude, longitude) for a collection of points.

    Parameters
    ----------
    polygon : shapely.geometry.polygon.Polygon instance
        Shapely Polygon containing coordinates in (longitude, latitude) convention.
    
    Returns
    -------
    polygon : shapely.geometry.polygon.Polygon instance
        Shapely Polygon containing coordinates in (latitude, longitude) convention.

    """
    polygon_vertices = polygon.exterior.coords[:]
    for idx, vertex in enumerate(polygon_vertices):
        polygon_vertices[idx] = vertex[::-1]
    polygon = shapely.geometry.Polygon(polygon_vertices)

    return polygon

In [8]:
# Convert the coordinates from (longitude, latitude) to (latitude, longitude).
districts_dataframe["geometry"] = districts_dataframe["geometry"].apply(lambda x: convert_coordinates(x))

### Computing the Centroids

* In this section we compute the centroids for each of the polygons representing the police districts.
* This can easily be done using GeoPandas.

In [9]:
centroids_dataframe = districts_dataframe.copy(deep=True)
centroids_dataframe["centroid"] = centroids_dataframe.centroid
centroids_dataframe

,district,geometry,centroid
0,BAYVIEW,"POLYGON ((37.765 -122.381, 37.762 -122.381, 37...",POINT (37.734 -122.390)
1,MISSION,"POLYGON ((37.769 -122.410, 37.769 -122.409, 37...",POINT (37.758 -122.423)
2,NORTHERN,"POLYGON ((37.808 -122.434, 37.808 -122.434, 37...",POINT (37.790 -122.432)
3,TENDERLOIN,"POLYGON ((37.786 -122.402, 37.774 -122.417, 37...",POINT (37.782 -122.413)
4,CENTRAL,"POLYGON ((37.807 -122.426, 37.807 -122.426, 37...",POINT (37.798 -122.409)
5,PARK,"POLYGON ((37.783 -122.440, 37.777 -122.438, 37...",POINT (37.764 -122.449)
6,RICHMOND,"POLYGON ((37.791 -122.441, 37.788 -122.441, 37...",POINT (37.778 -122.480)
7,INGLESIDE,"POLYGON ((37.749 -122.404, 37.748 -122.404, 37...",POINT (37.728 -122.432)
8,TARAVAL,"POLYGON ((37.708 -122.498, 37.708 -122.498, 37...",POINT (37.737 -122.482)
9,SOUTHERN,"POLYGON ((37.794 -122.392, 37.794 -122.392, 37...",POINT (37.776 -122.399)


### Plot the Results using the Google Maps API

* In this section we plot the results using the Google Maps API.

In [10]:
# Collate the district polygons into a single list.
polygons = [centroids_dataframe["geometry"][idx] for idx in range(len(districts_dataframe))]
polygon_centroids = [centroids_dataframe["centroid"][idx] for idx in range(len(districts_dataframe))]
polygon_vertices = [polygon.exterior.coords[:] for polygon in polygons]

In [11]:
# Populate the drawing and marker lists.
drawing_list = list()
marker_list = list()

for idx, vertices in enumerate(polygon_vertices):
    drawing_list.append(gmaps.Polygon(vertices, stroke_color="black", stroke_opacity=0.2,))
    centroid = polygon_centroids[idx].xy
    marker_list.append((centroid[0][0], centroid[1][0]))

In [12]:
# Plot the polygons using the Google Maps API.
fig = gmaps.figure()
drawings = gmaps.drawing_layer(drawing_list)
markers = gmaps.marker_layer(marker_list)
fig.add_layer(drawings)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

### Export the Centroid Information to CSV

* In this section we store the centroids that we computed along with the district geometries into a GeoPandas DataFrame for future usage.
* Note that the new data assumes the (latitude, longitude) information instead of (longitude, latitude).

In [19]:
path = "/Users/administrator/Documents/Projects/sf-crime-exploration/data/SFPD_District_Centroids.csv"
centroids_dataframe.to_csv(path, index=False)

### Load Data from CSV File

* In this section we load the CSV file that was just created to ensure that the data is accessible.
* Note that we need to convert from a Pandas DataFrame to a Pandas GeoDataFrame.

In [21]:
centroids_dataframe = pd.read_csv(path)
centroids_dataframe

,district,geometry,centroid
0,BAYVIEW,POLYGON ((37.76480022019017 -122.3809828136006...,POINT (37.73432830504337 -122.3896413590277)
1,MISSION,"POLYGON ((37.769317718404 -122.4095391743523, ...",POINT (37.75756983991986 -122.4226455774392)
2,NORTHERN,POLYGON ((37.80793010706978 -122.4337921715291...,POINT (37.78998465312932 -122.4317576421874)
3,TENDERLOIN,"POLYGON ((37.7862601311238 -122.4021713308171,...",POINT (37.78236656943267 -122.412761876948)
4,CENTRAL,"POLYGON ((37.80683854001055 -122.426120390961,...",POINT (37.79791152508096 -122.4091616666903)
5,PARK,"POLYGON ((37.7831382837805 -122.4395563094215,...",POINT (37.76434908040622 -122.4491077189602)
6,RICHMOND,"POLYGON ((37.79148984438553 -122.441268713802,...",POINT (37.77760526628989 -122.4796602993393)
7,INGLESIDE,POLYGON ((37.74858097286839 -122.4044981875426...,POINT (37.72788335066097 -122.4315859501734)
8,TARAVAL,POLYGON ((37.70810461000542 -122.4984166706208...,POINT (37.73663284365935 -122.4818302825414)
9,SOUTHERN,POLYGON ((37.79424680970543 -122.3918613881226...,POINT (37.77632134223039 -122.3988620645398)


In [24]:
# Convert the relevant columns to shapely geometric instances.
centroids_dataframe["geometry"] = centroids_dataframe["geometry"].apply(shapely.wkt.loads)
centroids_dataframe["centroid"] = centroids_dataframe["centroid"].apply(shapely.wkt.loads)

# Convert the Pandas DataFrame to a Pandas GeoDataFrame.
centroids_dataframe = gpd.GeoDataFrame(centroids_dataframe, geometry="geometry")

In [29]:
# Ensure that the GeoDataFrame contains the relevant data types.
print(type(centroids_dataframe["geometry"][0]))
print(type(centroids_dataframe["centroid"][0]))

<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.point.Point'>


This notebook looks good to go!

Note that this is the most recent version of center_of_mass.ipynb